In [1]:
import torch
import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

from animedata import AnimeRatings, AnimeRatingMatrix, MALAnime
from encoder import Encoder

In [2]:
# dataset = AnimeRatings()

In [3]:
# dataset.df.head()

In [4]:
ratings = AnimeRatings()

In [5]:
ratings.df.head()

,user_id,anime_id,score
0,690,2999,7
1,690,1946,7
2,690,1037,7
3,690,760,8
4,690,949,9


In [6]:
train_df, test_df = ratings.split_data(test_size=0.1)

In [7]:
train_mat = AnimeRatingMatrix(train_df, ratings.n_users, ratings.n_anime)
test_mat = AnimeRatingMatrix(test_df, ratings.n_users, ratings.n_anime)

In [8]:
# entire_im = InterMatrix(dataset.df, dataset.n_users, dataset.n_anime)

In [9]:
train_mat.matrix.size()

torch.Size([6000, 3000])

In [11]:
train_iter = train_mat.load_data(batch_size=256)

In [16]:
# net = Encoder(dataset.n_anime)
net = Encoder(ratings.n_anime, 500, dropout_rate=0.73)
# trainer = torch.optim.SGD(net.parameters(), lr=0.03)
# trainer = torch.optim.RMSprop(net.parameters(), lr = 0.01, weight_decay = 0.5)
trainer = torch.optim.Adam(net.parameters(), lr = 0.0001)
criterion = nn.MSELoss()
n_epoch = 500 

In [ ]:
losses = []

net.train() # turn out dropout

for epoch in range(n_epoch):
    
    total_error = 0
    cnt = 0
    
    
    for x in train_iter: 
        # print(x.size())
        # break
        tar = x
        pred = net(tar)
        
        # pred[inp == 0] = 0
        
        # loss = criterion(pred, inp)

        loss = net.masked_mse(pred, tar)
        # print(loss)
        # break
        
        
        # with torch.no_grad():
        #     mean_corrector = dataset.n_movies / (float(torch.sum(inp > 0) + 1e-10))
        #     total_error += torch.sqrt(loss.data * mean_corrector)
        #     # total_samples += x.shape[0]
        #     cnt += 1
        with torch.no_grad():
            total_error += torch.sqrt(loss)
            cnt += 1
            
        trainer.zero_grad()
        loss.backward()
        trainer.step()
        
            

    entire_loss = total_error / cnt

    if (epoch + 1) % 25 == 0:
        print(f'epoch {epoch + 1}, loss {entire_loss} '

In [13]:
with torch.no_grad():
    inp = train_mat.matrix[0]
    tar = test_mat.matrix[0] 
    pred = net(inp)
    
    

    
    for i, j in zip(tar, pred):
        if (i.item() != 0):
            print(i.item(), j.item())

    # pred[tar == 0] = 0
    # error = nn.functional.mse_loss(pred, tar)
    # error *= dataset.n_movies / (float(torch.sum(tar > 0)) + 1e-10)
    # print(torch.sqrt(error))
    
    mse = net.masked_mse(pred, tar)
    rmse = torch.sqrt(mse)
    print(rmse)
    

tensor(0.0035)
7.376284599304199
9.785659790039062
7.240827560424805
7.508613109588623
7.0461201667785645
yes
8.0
9.0
9.0
5.0
7.0
tensor(1.4420)


In [56]:
# Time for evaluation

net.eval() # turns off dropout

cnt = 0

sum_errors = 0

for i in range(train_mat.matrix.size()[0]):
    
    with torch.no_grad():
        inp = train_mat.matrix[i]
        tar = test_mat.matrix[i] 
        pred = net(inp)

        # pred[tar == 0] = 0

        # error = criterion(pred, tar)
        # error *= dataset.n_movies / (float(torch.sum(tar > 0)) + 1e-10)
        
        # sum_errors += torch.sqrt(error)


        mse = net.masked_mse(pred, tar)
        rmse = torch.sqrt(mse)

        sum_errors += rmse
        
        cnt += 1
    

print(sum_errors / cnt)    
    
# all_diffs = np.array(all_diffs)
# print(all_diffs.mean())

    # loss = net.get_loss(pred, x.float())
    
    # with torch.no_grad():
    #     total_error += loss * x.shape[0]
    #     total_samples += x.shape[0]

# test_loss = np.sqrt(total_error / total_samples)
# print(f'test loss: {test_loss}')        

# print(len(all_diffs)  

tensor(1.6226)


In [15]:
test_mat.inter_mat[0].size()

torch.Size([2977])

In [118]:
torch.ones(2977) * 7

tensor([7., 7., 7.,  ..., 7., 7., 7.])

In [41]:
def make_recommendations(user_id, k):
    
    inp = entire_im.inter_mat[user_id]
    pred = net(inp)
    
    pred[tar != 0] = 0
    
    values, indices = torch.sort(pred, descending=True)
    
    return indices[:k]
    

In [52]:
def see_user_ratings(user_id):
    rats = entire_im.inter_mat[user_id]
    
    values, indices = torch.sort(rats, descending=True)
    
    for v, idx in zip(values, indices):
        # print(v, idx)
        title = anime_df[anime_df['idx']==idx.item()]['title'].item()
        
        print(title, v.item())
    
    
    
    
    

In [17]:
a = train_mat.inter_mat[0]

In [55]:
a

tensor([0., 0., 0.,  ..., 0., 0., 0.])

tensor([10., 10., 10.,  9.,  9.,  9.,  9.,  9.,  9.,  9.])

In [65]:
min(indices)

tensor(0)

In [42]:
recs = make_recommendations(25, 15)

In [34]:
recs

tensor([2027, 1670, 2486, 2761, 2194, 2760, 2107, 2279, 1873, 2673, 2028,  606,
        1026, 2193, 2957])

In [54]:
for r in recs:
    title = anime_df[anime_df['idx']==r.item()]['title'].item()
    print(title)
    

Detective Conan Movie 13: The Raven Chaser
Detective Conan Movie 10: Requiem of the Detectives
Saiunkoku Monogatari 2nd Season
Golgo 13 (TV)
Detective Conan Movie 05: Countdown to Heaven
Osomatsu-san 2nd Season
Over Drive
Dragon Ball: Episode of Bardock
Gintama°
Diamond no Ace: Act II
InuYasha: Kanketsu-hen
Gintama'
Detective Conan Movie 15: Quarter of Silence
Oruchuban Ebichu
Detective Conan Movie 12: Full Score of Fear


In [22]:
anime_df = pd.read_csv('data/animes_clean.csv')

In [21]:
anime_df[anime_df['idx']==494]['title'].item()

NameError: name 'anime_df' is not defined

In [53]:
see_user_ratings(25)

Tsumiki no Ie 10.0
Detroit Metal City 10.0
Mononoke 10.0
Kanojo to Kanojo no Neko 10.0
Umineko no Naku Koro ni 10.0
Gravitation: Lyrics of Love 10.0
Vampire Knight 10.0
Gravitation 10.0
Hyakujitsu no Bara 10.0
Blame! 10.0
Canaan 10.0
Seirei no Moribito 10.0
Maria†Holic 10.0
Junjou Romantica 2 10.0
Tokyo Godfathers 10.0
Summer Wars 10.0
Kurozuka 10.0
Vampire Hunter D (2000) 10.0
Yami no Matsuei 10.0
Nekojiru-sou 10.0
Oruchuban Ebichu 10.0
Sweat Punch 10.0
Sensitive Pornograph 10.0
Neko no Ongaeshi 10.0
Petshop of Horrors 10.0
Yamato Nadeshiko Shichihenge♥ 10.0
Serial Experiments Lain 10.0
Bokura no 10.0
Panty & Stocking with Garterbelt 10.0
Claymore 10.0
Dogs: Bullets & Carnage 10.0
Haibane Renmei 10.0
Wolf's Rain 10.0
Kuragehime 10.0
Loveless 10.0
Michiko to Hatchin 10.0
Mononoke Hime 10.0
Hellsing 10.0
Sen to Chihiro no Kamikakushi 10.0
Tonari no Totoro 10.0
Howl no Ugoku Shiro 10.0
Kemonozume 10.0
Kuroshitsuji: Sono Shitsuji, Kougyou 10.0
Witch Hunter Robin 10.0
Blood+ 10.0
Byousoku 